<a href="https://colab.research.google.com/github/avocadopelvis/mental-load/blob/main/mental_load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
# import libraries
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

In [46]:
# use gpu if available 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [47]:
# load data
df = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/final.csv")
df = df.drop(df.columns[0], axis = 1)

# split data into training & testing data
train, test = train_test_split(df, test_size=0.15, random_state = 42)

In [48]:
test

,0,1,2,3,4,5,6,7,8,9,...,3831,3832,3833,3834,3835,3836,3837,3838,3839,label
1005,0.164014,0.166735,0.176727,0.196116,0.213293,0.219343,0.220973,0.227681,0.236188,0.237621,...,0.639941,0.849819,0.970012,0.898661,0.604743,0.176153,-0.223539,-0.453234,-0.465044,0
1078,0.002929,0.022080,0.076195,0.160300,0.230544,0.240186,0.187798,0.116244,0.067001,0.044995,...,0.224582,0.206815,0.172710,0.138659,0.113362,0.088653,0.052213,0.002051,-0.053440,1
67,-0.003629,-0.003397,-0.007976,-0.012933,-0.013669,-0.010251,-0.006219,-0.004303,-0.003996,-0.002730,...,-0.208054,-0.198667,-0.189793,-0.180948,-0.172425,-0.165161,-0.159735,-0.155701,-0.151847,2
867,0.048392,0.129867,0.240684,0.344691,0.380315,0.309278,0.155733,-0.010854,-0.128992,-0.184842,...,0.404173,0.373316,0.243216,0.077017,-0.056813,-0.132946,-0.175498,-0.215404,-0.247424,2
650,0.048999,0.049822,0.049900,0.050421,0.053110,0.058373,0.064244,0.068609,0.071240,0.072882,...,-0.035319,-0.034646,-0.029381,-0.024503,-0.024758,-0.029238,-0.032256,-0.028894,-0.018963,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,0.026601,0.036793,0.046180,0.051023,0.053659,0.058344,0.065527,0.072273,0.076169,0.078490,...,0.677449,0.286892,0.022926,-0.046464,-0.010843,-0.004386,-0.074311,-0.158254,-0.173879,1
1493,0.185808,0.179980,0.170797,0.159328,0.147434,0.135170,0.120580,0.103737,0.087794,0.074350,...,0.097044,0.086480,0.076435,0.065483,0.052853,0.039080,0.024595,0.009500,-0.004568,1
23,0.013307,0.015745,0.019292,0.022004,0.022628,0.021838,0.021186,0.021561,0.022855,0.024732,...,0.002636,0.001347,0.001552,0.002596,0.004134,0.005532,0.006023,0.005929,0.006296,1
1623,-0.111440,-0.111611,-0.110390,-0.108150,-0.105613,-0.102736,-0.099448,-0.096677,-0.094984,-0.093612,...,-0.552531,-0.449544,-0.356508,-0.284998,-0.233169,-0.193128,-0.153170,-0.100056,-0.026426,1


In [49]:
class SignalDataset(Dataset):
    def __init__(self, df):
      # get x and convert to tensor
      self.x = torch.tensor(df.iloc[:, 0:3840].values)
      # get y and convert to tensor
      self.y = torch.tensor(df["label"].values)
      # get number of samples
      self.n_samples = df.shape[0]

    def __len__(self):
      return self.n_samples

    def __getitem__(self, index):
      return self.x[index], self.y[index]

In [50]:
train_dataset = SignalDataset(train)
test_dataset = SignalDataset(test)

In [51]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True) #,num_workers = 2
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [52]:
# hyperparameters
input_size = 3840
hidden_size = 2500 
num_classes = 3
num_epochs = 20
batch_size = 64
learning_rate = 0.001

In [53]:
# FULLY CONNECTED NEURAL NETWORK

class NeuralNetwork(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNetwork, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out

model = NeuralNetwork(input_size, hidden_size, num_classes)
print(model)

NeuralNetwork(
  (l1): Linear(in_features=3840, out_features=2500, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=2500, out_features=3, bias=True)
)


In [54]:
# LOSS & OPTIMIZER
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

In [55]:
# TRAINING LOOP
n_total_steps = len(train_dataloader)

for epoch in range(num_epochs):
  for i, data in enumerate(train_dataloader, 0):
    # get the inputs and labels
    inputs, labels = data

    # forward
    outputs = model(inputs.float())
    loss = criterion(outputs, labels)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0:
      print(f'epoch {epoch+1} / {num_epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')

print("Finished Training")

epoch 1 / 20, step 100 / 179, loss = 1.3742
epoch 2 / 20, step 100 / 179, loss = 0.6643
epoch 3 / 20, step 100 / 179, loss = 0.6298
epoch 4 / 20, step 100 / 179, loss = 0.3719
epoch 5 / 20, step 100 / 179, loss = 0.1794
epoch 6 / 20, step 100 / 179, loss = 0.0374
epoch 7 / 20, step 100 / 179, loss = 0.0096
epoch 8 / 20, step 100 / 179, loss = 0.0427
epoch 9 / 20, step 100 / 179, loss = 0.0035
epoch 10 / 20, step 100 / 179, loss = 0.0020
epoch 11 / 20, step 100 / 179, loss = 0.2903
epoch 12 / 20, step 100 / 179, loss = 0.0073
epoch 13 / 20, step 100 / 179, loss = 0.4878
epoch 14 / 20, step 100 / 179, loss = 0.0271
epoch 15 / 20, step 100 / 179, loss = 0.0194
epoch 16 / 20, step 100 / 179, loss = 0.0003
epoch 17 / 20, step 100 / 179, loss = 0.0007
epoch 18 / 20, step 100 / 179, loss = 0.0016
epoch 19 / 20, step 100 / 179, loss = 0.0001
epoch 20 / 20, step 100 / 179, loss = 0.0000
Finished Training


In [56]:
# TESTING
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for inputs, labels in test_dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs.float())

    # value, index (We don't need value)
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc}')

accuracy = 41.26984126984127
